In [1]:
import pandas as pd
from utils import MysqlConnector
conn = MysqlConnector()

In [5]:
conn.execute("""USE dw;""")
conn.execute("""DROP TABLE IF EXISTS factory_dim;""")
conn.execute("""CREATE TABLE factory_dim (
  factory_sk INT NOT NULL AUTO_INCREMENT PRIMARY KEY
, factory_code INT
, factory_name CHAR(30)
, factory_street_address CHAR(50)
, factory_zip_code INT(5)
, factory_city CHAR(30)
, factory_state CHAR(2)
, effective_date DATE
, expiry_date DATE
)
;""")
conn.execute("""DROP TABLE IF EXISTS production_fact;""")
conn.execute("""CREATE TABLE production_fact (
  product_sk INT 
, production_date_sk INT
, factory_sk INT 
, production_quantity INT
) 
;""")

In [6]:
conn.execute("""USE source;""")
conn.execute("""DROP TABLE IF EXISTS factory_master;""")
conn.execute("""CREATE TABLE factory_master
( factory_code INT
, factory_name CHAR(30)
, factory_street_address CHAR(50)
, factory_zip_code INT(5)
, factory_city CHAR(30)
, factory_state CHAR(2) )
;""")
conn.execute("""INSERT INTO factory_master VALUES
  ( 1, 'First Factory', '11111 Lichtman St.', 17050,  'Mechanicsburg', 'PA' )
, ( 2, 'Second Factory', '22222 Stobosky Ave.', 17055, 'Pittsburgh', 'PA' )
, ( 3, 'Third Factory', '33333 Fritze Rd.', 17050, 'Mechanicsburg', 'PA' )
, ( 4, 'Fourth Factory', '44444 Jenzen Blvd.', 17055, 'Pittsburgh', 'PA' )
, ( 5, 'Fifth Factory', '55555 Jachin Blvd.', 2200, 'Austin', 'TX' )
, ( 6, 'Sixth Factory', '66666 Dakota Center', 55, 'Oregon', 'OR' )
, ( 7, 'Seven Factory', '77777 San Mateo', 105, 'Phoenix', 'AZ' )
;
""")

In [7]:
conn.execute("""USE dw;""")
conn.execute("""INSERT INTO factory_dim
SELECT
  NULL
, factory_code
, factory_name
, factory_street_address
, factory_zip_code
, factory_city
, factory_state
, CURRENT_DATE
, '9999-12-31'
FROM source.factory_master
;""")

In [3]:
conn.execute("""USE dw;""")
conn.execute("""DROP TABLE IF EXISTS factory_stg;""")
conn.execute("""CREATE TABLE factory_stg 
( factory_code INT
, factory_name CHAR(30)
, factory_street_address CHAR(50)
, factory_zip_code INT(5)
, factory_city CHAR(30)
, factory_state CHAR(2) )
;""")

In [4]:
conn.execute("""USE SOURCE;""")
conn.execute("""DROP TABLE IF EXISTS daily_production;""")
conn.execute("""CREATE TABLE daily_production
( product_code INT
, production_date DATE
, factory_code INT 
, production_quantity INT )
;""")
conn.execute("""INSERT INTO daily_production VALUES
  (1, CURRENT_DATE, 4, 100 )
, (2, CURRENT_DATE, 3, 200 )
, (3, CURRENT_DATE, 2, 300 )
, (4, CURRENT_DATE, 1, 400 )
, (1, CURRENT_DATE, 1, 400 )
, (2, CURRENT_DATE, 2, 300 )
, (3, CURRENT_DATE, 3, 200 )
, (4, CURRENT_DATE, 4, 100 )
, (5, CURRENT_DATE, 1, 500 )
, (6, CURRENT_DATE, 2, 600 )
, (7, CURRENT_DATE, 3, 700 )
, (8, CURRENT_DATE, 4, 800 )
, (8, CURRENT_DATE, 1, 900 )
, (9, CURRENT_DATE, 2, 1000 )
, (10, CURRENT_DATE, 3, 1200 )
, (9, CURRENT_DATE, 4, 1200 )
, (6, CURRENT_DATE, 3, 2300 )
, (5, CURRENT_DATE, 4, 1400 )
, (4, CURRENT_DATE, 1, 4500 )
, (9, CURRENT_DATE, 2, 3600 )
, (10, CURRENT_DATE, 3, 2600 )
, (5, CURRENT_DATE, 4, 1400 )
, (8, CURRENT_DATE, 5, 900 )
, (9, CURRENT_DATE, 6, 1000 )
, (10, CURRENT_DATE, 6, 1200 )
, (9, CURRENT_DATE, 6, 1200 )
, (6, CURRENT_DATE, 5, 2300 )
, (5, CURRENT_DATE, 5, 1400 )
, (4, CURRENT_DATE, 5, 4500 )
, (9, CURRENT_DATE, 6, 3600 )
, (10, CURRENT_DATE, 6, 2600 )
, (5, CURRENT_DATE, 6, 1400 )
, (9, CURRENT_DATE, 7, 13600 )
, (10, CURRENT_DATE, 7, 22600 )
, (5, CURRENT_DATE, 7, 14300 )
, (8, CURRENT_DATE, 7, 9200 )
, (9, CURRENT_DATE, 7, 10200 )
, (10, CURRENT_DATE, 7, 12200 )
, (9, CURRENT_DATE, 7, 11200 )
, (6, CURRENT_DATE, 7, 23100 )
, (5, CURRENT_DATE, 7, 14100 )
, (4, CURRENT_DATE, 7, 14500 )
, (9, CURRENT_DATE, 7, 36300 )
, (10, CURRENT_DATE, 4, 26200 )
, (5, CURRENT_DATE, 2, 1400 )
;
""")

In [8]:
conn.execute("""USE dw;""")
conn.execute("""TRUNCATE factory_stg;""")

In [9]:
df = pd.read_csv('assets/FACTORY.csv')
df.head()

,factory_code,factory_name,factory_street_address,factory_zip_code,factory_city,factory_state
0,2,Second Factory,24242 Bunty La.,17055,Pittsburgh,PA
1,3,Third Factory,33333 Fritze Rd.,17050,Mechanicsburg,PA
2,5,Fifth Factory,55555 Jachin Blvd.,2200,Austin,TX
3,6,Sixth Factory,66666 Dakota Center,55,Oregon,OR
4,7,Seven Factory,77777 San Mateo,105,Phoenix,AZ


In [10]:
conn.execute("USE dw;")
conn.batch_insert('factory_stg', df.columns.tolist(), df.values.tolist())

In [11]:
conn.execute("""UPDATE factory_dim a, factory_stg b
SET 
  a.factory_name = b.factory_name
, a.factory_street_address = b.factory_street_address
, a.factory_zip_code = b.factory_zip_code
, a.factory_city = b.factory_city
, a.factory_state = b.factory_state
WHERE a.factory_code = b.factory_code
;""")
conn.execute("""INSERT INTO factory_dim
SELECT
  NULL
, factory_code
, factory_name
, factory_street_address
, factory_zip_code
, factory_city
, factory_state 
, CURRENT_DATE
, '9999-12-31'
FROM factory_stg
WHERE factory_code NOT IN (
SELECT y.factory_code 
FROM factory_dim x, factory_stg y 
WHERE x.factory_code = y.factory_code )
;""")   
conn.execute("""INSERT INTO production_fact
SELECT
  b.product_sk
, c.date_sk
, d.factory_sk
, production_quantity
FROM
  source.daily_production a
, product_dim b
, date_dim c
, factory_dim d
WHERE a.product_code = b.product_code
AND a.factory_code = d.factory_code
;""")            

In [12]:
conn.execute("""USE source;""")
conn.execute("""INSERT INTO daily_production VALUES
  ( 1, CURRENT_DATE, 4, 100 )
, ( 2, CURRENT_DATE, 3, 200 )
, ( 3, CURRENT_DATE, 2, 300 )
, ( 4, CURRENT_DATE, 1, 400 )
, ( 1, CURRENT_DATE, 1, 400 )
, ( 2, CURRENT_DATE, 2, 300 )
, ( 3, CURRENT_DATE, 3, 200 )
, ( 4, CURRENT_DATE, 4, 100 )
, ( 6, CURRENT_DATE, 2, 700 )
, ( 7, CURRENT_DATE, 1, 800 )
, ( 8, CURRENT_DATE, 1, 800 )
, ( 9, CURRENT_DATE, 2, 400 )
, ( 10, CURRENT_DATE, 3, 200 )
, ( 4, CURRENT_DATE, 4, 300 )
, ( 3, CURRENT_DATE, 2, 400 )
, ( 2, CURRENT_DATE, 1, 500 )
, ( 10, CURRENT_DATE, 1, 500 )
, ( 2, CURRENT_DATE, 2, 300 )
, ( 4, CURRENT_DATE, 3, 200 )
, ( 5, CURRENT_DATE, 4, 100 )
, (8, CURRENT_DATE, 5, 900 )
, (9, CURRENT_DATE, 6, 1000 )
, (10, CURRENT_DATE, 6, 1200 )
, (9, CURRENT_DATE, 6, 1200 )
, (6, CURRENT_DATE, 5, 2300 )
, (5, CURRENT_DATE, 5, 1400 )
, (4, CURRENT_DATE, 5, 4500 )
, (9, CURRENT_DATE, 6, 3600 )
, (10, CURRENT_DATE, 6, 2600 )
, (5, CURRENT_DATE, 6, 1400 )
, ( 5, CURRENT_DATE, 4, 100 )
, (8, CURRENT_DATE, 5, 900 )
, (9, CURRENT_DATE, 6, 1000 )
, (10, CURRENT_DATE, 6, 1200 )
, (9, CURRENT_DATE, 6, 1200 )
, (6, CURRENT_DATE, 5, 2300 )
, (5, CURRENT_DATE, 5, 1400 )
, (4, CURRENT_DATE, 5, 4500 )
, (9, CURRENT_DATE, 6, 3600 )
, (10, CURRENT_DATE, 6, 2600 )
, (5, CURRENT_DATE, 6, 1400 )
, ( 5, CURRENT_DATE,7, 100 )
, (8, CURRENT_DATE, 1, 900 )
, (9, CURRENT_DATE, 7, 1000 )
, (10, CURRENT_DATE, 7, 1200 )
, (9, CURRENT_DATE, 7, 1200 )
, (6, CURRENT_DATE, 6, 2300 )
, (5, CURRENT_DATE, 4, 1400 )
, (4, CURRENT_DATE, 1, 4500 )
, (9, CURRENT_DATE, 1, 3600 )
, (10, CURRENT_DATE, 1, 2600 )
, (5, CURRENT_DATE, 7, 1400 )
, ( 5, CURRENT_DATE, 7, 100 )
, (8, CURRENT_DATE, 7, 900 )
, (9, CURRENT_DATE, 7, 1000 )
, (10, CURRENT_DATE, 7, 1200 )
, (9, CURRENT_DATE, 6, 1200 )
, (6, CURRENT_DATE, 4, 2300 )
, (5, CURRENT_DATE, 7, 1400 )
, (4, CURRENT_DATE, 7, 4500 )
, (9, CURRENT_DATE, 7, 3600 )
, (10, CURRENT_DATE, 7, 2600 )
, (5, CURRENT_DATE, 5, 5400 )
;
""")

Confirmar a criação da tabela fato PRODUCTION_FACT e apresentar umgráfico das fábricas que mais produziram e um relatório (contendonome, endereço, cidade, estado e quantidade produzida).

In [20]:
conn.execute("USE dw;")
result = conn.fetch('''
SELECT factory_name, factory_street_address, factory_city, factory_state, sum(production_quantity) total from production_fact pf
JOIN product_dim pd ON pf.product_sk = pd.product_sk    
JOIN factory_dim fd ON pf.factory_sk = fd.factory_sk    
JOIN date_dim dd ON pf.production_date_sk = dd.date_sk    
group by factory_name, factory_street_address, factory_city, factory_state
order by total desc
''')
pd.DataFrame(result)

,0,1,2,3,4
0,Seven Factory,77777 San Mateo,Phoenix,AZ,198523500
1,Fourth Factory,44444 Jenzen Blvd.,Pittsburgh,PA,34164000
2,Sixth Factory,66666 Dakota Center,Oregon,OR,12045000
3,Fifth Factory,55555 Jachin Blvd.,Austin,TX,9964500
4,Third Factory,33333 Fritze Rd.,Mechanicsburg,PA,7884000
5,Second Factory,24242 Bunty La.,Pittsburgh,PA,7884000
6,First Factory,11111 Lichtman St.,Mechanicsburg,PA,7336500
